In [ ]:
import re
from typing import Dict, Any, Union

def parse_css_variables(css_content: str) -> Dict[str, str]:
    """Parse CSS content and extract variable definitions."""
    # Remove comments
    css_content = re.sub(r'/\*.*?\*/', '', css_content, flags=re.DOTALL)
    
    # Remove newlines and extra whitespace
    css_content = re.sub(r'\s+', ' ', css_content)
    
    # Find the :root block
    root_match = re.search(r':root\s*{([^}]+)}', css_content)
    if not root_match:
        raise ValueError("No :root block found in CSS")
        
    root_content = root_match.group(1)
    
    # Parse variable definitions
    pattern = r'--([^:]+):\s*([^;]+);'
    matches = re.findall(pattern, root_content)
    
    variables = {}
    for name, value in matches:
        name = name.strip()
        value = value.strip()
        variables[name] = value
        
    return variables

def extract_var_name(var_ref: str) -> str:
    """Extract the variable name from a var() reference."""
    match = re.match(r'var\(--(.*?)\)', var_ref)
    return match.group(1) if match else var_ref.lstrip('-')

def resolve_variable_reference(var_ref: str, variables: Dict[str, str], visited: set = None) -> str:
    """Resolve a CSS variable reference to its final value."""
    if visited is None:
        visited = set()
    
    var_name = extract_var_name(var_ref)
    
    if var_name in visited:
        raise ValueError(f"Circular reference detected for variable: {var_name}")
    
    visited.add(var_name)
    
    if var_name not in variables:
        raise ValueError(f"Variable not found: {var_name}")
    
    value = variables[var_name]
    
    # If value contains another variable reference, resolve it recursively
    if 'var(--' in value:
        while 'var(--' in value:
            var_match = re.search(r'var\(--([^)]+)\)', value)
            if var_match:
                referenced_var = var_match.group(1)
                try:
                    resolved_value = resolve_variable_reference(f"var(--{referenced_var})", variables, visited.copy())
                    value = value.replace(f"var(--{referenced_var})", resolved_value)
                except ValueError as e:
                    raise ValueError(f"Failed to resolve {referenced_var}: {str(e)}")
    
    return value

def trace_variable_chain(var_ref: str, variables: Dict[str, str]) -> list:
    """Trace the chain of variable references."""
    chain = []
    current_var = extract_var_name(var_ref)
    
    while True:
        if current_var not in variables:
            break
            
        value = variables[current_var]
        chain.append((current_var, value))
        
        if 'var(--' in value:
            var_match = re.search(r'var\(--([^)]+)\)', value)
            if var_match:
                current_var = var_match.group(1)
            else:
                break
        else:
            break
    
    return chain

def resolve_nested_structure(value: Any, variables: Dict[str, str], debug: bool = False, path: str = "") -> Any:
    """Recursively resolve values in nested structures."""
    if isinstance(value, dict):
        return {k: resolve_nested_structure(v, variables, debug, f"{path}.{k}" if path else k) 
                for k, v in value.items()}
    elif isinstance(value, list):
        return [resolve_nested_structure(item, variables, debug, f"{path}[{i}]")
                for i, item in enumerate(value)]
    elif isinstance(value, str) and value.startswith('var(--'):
        try:
            if debug:
                print(f"\nResolving {path}:")
                chain = trace_variable_chain(value, variables)
                for var, val in chain:
                    print(f"  --{var} → {val}")
            return resolve_variable_reference(value, variables)
        except ValueError as e:
            print(f"Warning: Could not resolve {path}: {e}")
            return value
    else:
        return value

def resolve_theme_variables(theme: Dict[str, Any], css_content: str, debug: bool = False) -> Dict[str, Any]:
    """Resolve all CSS variables in a theme dictionary to their final values."""
    try:
        variables = parse_css_variables(css_content)
        
        if debug:
            print("\nFound CSS variables:")
            for name, value in sorted(variables.items()):
                print(f"--{name}: {value}")
            print()
        
        return resolve_nested_structure(theme, variables, debug)
        
    except Exception as e:
        print(f"Error parsing CSS: {str(e)}")
        return theme

# Example usage
if __name__ == "__main__":
    TABLE_THEME = {
        'colors': {
            'header_bg': 'var(--table-header-bg)',
            'header_text': 'var(--table-header-text)',
            'cell_bg': 'var(--table-cell-bg)',
            'cell_text': 'var(--table-cell-text)',
            'border': 'var(--table-border)',
            'highlight': 'var(--table-highlight-bg)',
            'success': 'var(--color-status-success)',
            'danger': 'var(--color-status-danger)',
            'qtoq_bg': 'var(--table-qtoq-bg)',
        },
        'typography': {
            'font_family': 'var(--font-family-base)',
            'font_size': 'var(--table-font-size)',
            'header_weight': 'var(--font-weight-semibold)',
            'bold_weight': 'bold',
            'normal_weight': 'normal'
        },
        'spacing': {
            'cell_padding': 'var(--table-cell-padding-x)',
            'header_padding': 'var(--table-header-padding)',
        },
        'layout': {
            'white_space': 'normal',
            'min_width': '100%',
            'overflow_x': 'auto',
            'text_align': {
                'left': 'left',
                'center': 'center'
            },
            'vertical_align': {
                'top': 'top',
                'bottom': 'bottom'
            },
            'border': {
                'width': '1px',
                'style': 'solid',
                'none': 'none'
            }
        },
        'special_rows': {
            'top': {
                'backgroundColor': 'var(--table-highlight-bg)',
                'fontWeight': 'normal'
            },
            'market': {
                'backgroundColor': 'var(--table-highlight-bg)',
                'fontWeight': 'bold'
            }
        }
    }
    
    # Read CSS content from file
    with open('_design-tokens.css', 'r') as f:
        css_content = f.read()
    
    # Resolve all variables in the theme with debug output
    resolved_theme = resolve_theme_variables(TABLE_THEME, css_content, debug=True)
    
    # Print the resolved theme with nice formatting
    import json
    print("\nResolved Theme:")
    print(json.dumps(resolved_theme, indent=2))